<a href="https://colab.research.google.com/github/afaale/ML/blob/ML/Bert_transfer_lr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **model1**

In [ ]:
!pip install transformers
!pip install datasets

In [10]:
from transformers import pipeline
import numpy as np

In [ ]:
unmasker = pipeline('fill-mask', model='bert-base-cased')
unmasker("King is a [MASK].")


In [ ]:
from datasets import load_dataset

dataset = load_dataset("glue", "cola")
dataset_train = dataset["train"]  
dataset_test = dataset['test']


In [14]:
dataset_test

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 1063
})

In [ ]:
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam

# Load and compile our model
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


def tokenize_dataset(data):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(data["sentence"])


dataset_train = dataset_train.map(tokenize_dataset)
dataset_test = dataset_test.map(tokenize_dataset)

In [17]:
dataset_train[0]

{'sentence': "Our friends won't buy this analysis, let alone the next one we propose.",
 'label': 1,
 'idx': 0,
 'input_ids': [101,
  2256,
  2814,
  2180,
  1005,
  1056,
  4965,
  2023,
  4106,
  1010,
  2292,
  2894,
  1996,
  2279,
  2028,
  2057,
  16599,
  1012,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tf_dataset = model.prepare_tf_dataset(dataset_train, batch_size=16, shuffle=True, tokenizer=tokenizer)
model.compile(optimizer=Adam(3e-5), metrics=['accuracy'])  # No loss argument!

history = model.fit(tf_dataset, epochs=1)

In [ ]:
question_answerer = pipeline("question-answering",  "Rocketknight1/distilbert-base-uncased-finetuned-squad", framework="tf")


In [ ]:
context = """The dominant sequence transduction models are based on complex recurrent or convolutional 
neural networks in an encoder-decoder configuration. The best performing models also connect the encoder
and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, 
based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on 
two machine translation tasks show these models to be superior in quality while being more parallelizable 
and requiring significantly less time to train."""
question = "What are Transformers?"
question_answerer(context, question, )

# model 2

In [ ]:
!pip install transformers
!pip install datasets

In [24]:
import numpy as np

In [ ]:
from datasets import load_dataset

dataset = load_dataset("glue", "cola")
dataset_train = dataset["train"]  
dataset_test = dataset['test']


In [41]:
tokenized_data = tokenizer(dataset_train["sentence"], return_tensors="np", padding=True)
tokenized_data = dict(tokenized_data)
labels = np.array(dataset_train["label"])  # Label is already an array of 0 and 1


In [43]:
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam

# Load and compile our model
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
# Lower learning rates are often better for fine-tuning transformers
model.compile(optimizer=Adam(3e-5))  # No loss argument!

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
tokenized_data_test = dict(tokenizer(dataset_test["sentence"], return_tensors="np", padding=True))
labels_test = np.array(dataset_test["label"])  # Label is already an array of 0 and 1


model.evaluate(tokenized_data_test)

34/34 [==============================] - 9s 9ms/step - loss: 0.0000e+00


0.0

# model3

In [36]:
from transformers import AutoTokenizer, TFAutoModelForCausalLM

# You can, of course, use your own username and model name here 
# once you've pushed your model using the code above!
checkpoint = "Rocketknight1/distilgpt2-finetuned-wikitext2"
model = TFAutoModelForCausalLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at Rocketknight1/distilgpt2-finetuned-wikitext2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [17]:
def tokenize_dataset(data):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(data["sentence"])


dataset_train = dataset_train.map(tokenize_dataset)
dataset_test = dataset_test.map(tokenize_dataset)

In [31]:
history = model.fit(tokenized_data, labels)


ValueError: ignored

In [13]:
def prompt(test_sentence):
    tokenized = tokenizer(test_sentence, return_tensors="np")
    outputs = model.generate(**tokenized, max_length=32)
    print('\n')
    print(tokenizer.decode(outputs[0]))

test_sentence = "A confusion matrix is"
prompt(test_sentence)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




A confusion matrix is a matrix of two elements. The first element is the first element, the second element is the second element. The first element is the second element. The second element is the third element. The third element is the fourth element. The fourth element is the fourth element. The fourth element is the fourth
